## WIFI Networking
I really like the ESP32 becase it comes with "Batteries included" as far as networking is concearned. 
So I wanted to be able to emulate this as well:

First we define the emulator:

In [16]:
#pragma cling add_include_path("ArduinoCore-API/api")
#pragma cling add_include_path("ArduinoCore-Linux/cores/arduino")
#pragma cling add_library_path("build") 
#pragma cling load("libarduino_emulator") 

This is a standard ESP32 setup which is using WIFI:

In [17]:
#include "Arduino.h"
#include "WiFi.h"

void setup() {
    Serial.begin(115200);
    Logger.setLogger(Serial,Warning);
    delay(10);

    Serial.println();
    Serial.println();
    Serial.print("Connecting to Network...");

    WiFi.begin("ssid not relevant", "password not relevant");
    while (WiFi.status() != WL_CONNECTED) {
        delay(500);
        Serial.print(".");
    }

    Serial.println();
    Serial.println("WiFi connected");
    Serial.print("IP address: ");
    Serial.println(WiFi.localIP());
}

setup();



Connecting to Network...
WiFi connected
IP address: 172.22.0.2


Now we can try out a simple Http request using the WiFiClient.
First we connect to an address:

In [11]:
#include "WiFiClient.h"

WiFiClient client;
if (!client.connect("134.209.234.6", 80)) {
    Serial.println("connection failed");
}

client.connected()==true

true

In [12]:
client.connected()==true

true

We send the http request to the server:

In [13]:
client.println("GET / HTTP/1.0");
client.println("Connection: close");
client.println();


... and we process the result:

In [14]:
while (client.connected()) {
  String line = client.readStringUntil('\n');
  Serial.println(line);
  if (line == "\r") {
    Serial.println("==>headers received<==");
    break;
  }
}
// if there are incoming bytes available
// from the server, read them and print them:
while (client.available()) {
  char c = client.read();
  Serial.write(c);
}



HTTP/1.1 301 Moved Permanently
Server: nginx/1.17.6
Date: Fri, 11 Dec 2020 07:27:31 GMT
Content-Type: text/html
Content-Length: 169
Connection: close
Location: https://software.pschatzmann.ch/

==>headers received<==
<html>
<head><title>301 Moved Permanently</title></head>
<body>
<center><h1>301 Moved Permanently</h1></center>
<hr><center>nginx/1.17.6</center>
</body>
</html>


Finally we close the connections:

In [15]:
client.stop();